# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

salaries.columns = [x.lower().replace(' ','_') for x in salaries.columns]

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
salaries.isnull().sum()

name                     0
job_titles               0
department               0
full_or_part-time        0
salary_or_hourly         0
typical_hours        25161
annual_salary         8022
hourly_rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
salaries.salary_or_hourly.value_counts()


salary_or_hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
salaries.groupby('department')['salary_or_hourly'].count()

department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

H0: the average is equal to 30

H1: the average is not equal to 30

confianza: 95%

In [7]:
from scipy.stats import ttest_1samp  
from scipy.stats import t

In [8]:
df = salaries[salaries['salary_or_hourly'] == 'Hourly']
df.head()

,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
10,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,NaN,2.65
18,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68


In [61]:
mu = 30 # media población

mu_hat = df.hourly_rate.mean() # media muestral

std_hat = df.hourly_rate.std() # std muestral

n = len(df) # tamaño muestral

dof = n-1 #degrees of freedom

In [62]:
t_test = (mu_hat - mu) / (std_hat/np.sqrt(n)) # diferencia entre la media de la muestra y la media poblacional en unidades de error estándar.

p_value = stats.t(df=dof).cdf(t_test) # p_valor, probabilidad correspondiente al estadistico 

result = stats.t.sf(np.abs(t_test), df=dof) * 2 # Para cuando es two-sided
result


4.3230240486229894e-92

In [63]:
# Con formula
test = ttest_1samp(df.hourly_rate,
                   30,
                   alternative='two-sided')

test.pvalue

4.3230240486229894e-92

In [64]:
alpha = 0.05

if test.pvalue < alpha:
    print('Rechazamos hipótesis nula. Media significativamente diferente a 30/h')
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")

Rechazamos hipótesis nula. Media significativamente diferente a 30/h


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

H0: police salaries mean higher than $86000/year

H1: police salaries mean equal or lower than $86000/year

In [73]:
df_pol = salaries[(salaries['salary_or_hourly'] == 'Salary') & (salaries['department'] == 'POLICE') | (salaries['department'] == 'POLICE BOARD')]

In [80]:
df_pol.head()

,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
11,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,NaN,84054.0,NaN
12,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN


In [75]:
mu = 86_000 # media población

mu_hat = df_pol.annual_salary.mean() # media muestral

std_hat = df_pol.annual_salary.std() # std muestral

n = len(df_pol) # tamaño muestral

dof = n-1 #degrees of freedom

In [96]:
# Con formula
test = ttest_1samp(df_pol.annual_salary,
                   mu,
                   alternative='less')

test.pvalue

0.9989693542994336

In [98]:
alpha = 0.05

if test.pvalue < alpha:
    print('Rechazamos la hipótesis nula. Hay evidencia suficiente para afirmar que los salarios son mayores a $86000/año.')
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula. No podemos concluir que los salarios sean mayores a $86000/año.")

No hay suficiente evidencia para rechazar la hipótesis nula. No podemos concluir que los salarios sean mayores a $86000/año.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [114]:
cross = pd.crosstab(salaries['department'], salaries['salary_or_hourly'])
trabajo = cross['Hourly']
trabajos = trabajo.idxmax()
trabajos

'STREETS & SAN'

In [113]:
df.department.value_counts().sort_values(ascending=False)[:1]

department
STREETS & SAN    1862
Name: count, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

H0: El salario es igual o mayor que 35/h

H1: El salario es menor que 35/h

In [127]:
df_most = salaries[(salaries['department'] == 'STREETS & SAN') & (salaries['salary_or_hourly'] == 'Hourly')]

In [130]:
# Con formula
test = ttest_1samp(df_most.hourly_rate,
                   35,
                   alternative='less')

test.pvalue

1.6689265282353859e-21

In [132]:
alpha = 0.05

if test.pvalue < alpha:
    print('Rechazamos la hipótesis nula. Hay evidencia suficiente para afirmar que los salarios son menores a $35/h.')
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula. No podemos concluir que los salarios sean menores a $35/h.")

Rechazamos la hipótesis nula. Hay evidencia suficiente para afirmar que los salarios son menores a $35/h.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [139]:
from scipy.stats import sem

In [167]:
interval = t.interval(0.95,      
            len(df.hourly_rate)-1,
            loc=df.hourly_rate.mean(),
            scale=sem(df.hourly_rate)
            )

interval = [round(i,2) for i in interval]
interval

[32.52, 33.05]

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [168]:
interval = t.interval(0.95,      
            len(df_pol.annual_salary)-1,
            loc=df_pol.annual_salary.mean(),
            scale=sem(df_pol.annual_salary)
            )

interval = [round(i,2) for i in interval]
interval

[86177.02, 86795.7]

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [178]:
from statsmodels.stats.proportion import proportions_ztest


In [184]:
salaries['salary_or_hourly'].value_counts()

salary_or_hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

In [193]:
# H0 no es igual
# H1 si es igual

z_score, p_value= proportions_ztest(count = salaries['salary_or_hourly'].value_counts()[1],
                                    nobs = len(salaries),
                                    value = 0.25)

z_score, p_value

/var/folders/h7/xhgpgb3d0s55331krx9jbrv80000gn/T/ipykernel_2464/3427396043.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  z_score, p_value= proportions_ztest(count = salaries['salary_or_hourly'].value_counts()[1],


(-3.5099964213703005, 0.0004481127249057967)

In [196]:
alpha = 0.05

# the number of hourly workers in the City of Chicago is significantly different from 25%

if p_value < alpha:
    print('Rechazamos la hipótesis nula. El número de trabajadores por hora en Chicago no es significativamente diferente del 25%')
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula. El número de trabajadores por hora en Chicago si es significativamente diferente del 25%")

Rechazamos la hipótesis nula. El número de trabajadores por hora en Chicago no es significativamente diferente del 25%
